In [1]:
#Imports
import cv2
import numpy as np
import glob

In [2]:
#Define checkerboard size
checkerboard_size = (8,6)
frame_size = (2208, 1242)

In [3]:
#termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [4]:
#Prepare object points
object_points = np.zeros((checkerboard_size[0] * checkerboard_size[1], 3), np.float32)
object_points[:,:2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1,2)

In [5]:
#Arrays to store object points and image points from all images
object_points_list = [] #3d points real world space
image_points_list = [] #2d points image plain

In [6]:
#Path to the images
images = glob.glob("/Users/davidschaupp/Documents/GitHub/Vision/Camera_Calibration/Calibration/Checkerboard_Images/*.png")
images = glob.glob("/Users/davidschaupp/Downloads/cam_4_calibration_images/*.png")

In [7]:
for image in images:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #detect corners
    ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)

    if ret == True:
        object_points_list.append(object_points)
        corners_2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)
        image_points_list.append(corners)

        #Draw and display the corners

        cv2.drawChessboardCorners(img, checkerboard_size, corners_2, ret)
        cv2.imshow("img", img)
        cv2.waitKey(1000)


cv2.destroyAllWindows()

In [8]:
#Calibration
ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(object_points_list, image_points_list, frame_size, None, None)

In [9]:
print(f"Camera calibrated: {ret}")
print(f"\n Camera Matrix: \n {cameraMatrix}")
print(f"\n Disortion Parameters: \n {dist}")
print(f"\n Rotation Vectors: \n {rvecs}")
print(f"\nTranslation Vectors: \n {tvecs}")

Camera calibrated: 2.1766772560802887

 Camera Matrix: 
 [[9.34315064e+02 0.00000000e+00 1.09837116e+03]
 [0.00000000e+00 9.17677485e+02 6.36129872e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

 Disortion Parameters: 
 [[-0.13330761  0.30729155 -0.02002784  0.00569651 -0.27627372]]

 Rotation Vectors: 
 [array([[ 0.14380487],
       [ 0.0836261 ],
       [-0.04802386]]), array([[0.12539606],
       [0.43903943],
       [0.26253695]]), array([[-0.7396445 ],
       [ 0.64026744],
       [ 1.39407877]]), array([[ 0.35761921],
       [ 0.4886955 ],
       [-1.48953462]]), array([[-0.66303406],
       [-0.72063988],
       [-1.44595831]]), array([[0.0649865 ],
       [0.0298982 ],
       [0.06626009]]), array([[-0.71737461],
       [ 0.65207799],
       [ 1.40794941]]), array([[-0.18792132],
       [-0.15410522],
       [-1.48593312]]), array([[ 0.048834  ],
       [ 0.21430579],
       [-1.50983079]]), array([[-1.16514595],
       [ 0.94013655],
       [ 1.22300172]]), array([[-0.

In [9]:
#Get new camera matrix
img = cv2.imread("/Users/davidschaupp/Downloads/cam_4_calibration_images/left000704.png")
h, w = img.shape[:2]
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

In [13]:
new_camera_matrix

array([[1.36118262e+03, 0.00000000e+00, 1.15754649e+03],
       [0.00000000e+00, 8.27968628e+02, 7.19270583e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [12]:
#Undistort
dst = cv2.undistort(img, cameraMatrix, dist, None, new_camera_matrix)
#crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite("/Users/davidschaupp/Downloads/calibration_result.png", dst)

True

In [31]:
"""
#Undistort with remapping
mapx, mapy = cv2.initUndistortRectifyMap(cameraMatrix, dist, None, new_camera_matrix, (w,h), 5)
dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
#crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite("/Users/davidschaupp/Documents/GitHub/Vision/Camera_Calibration/Calibration/Checkerboard_Images/calibration_result_mapped.png", dst)
"""


True